In [1]:
import gym
import numpy as np
import itertools
import random
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

Using TensorFlow backend.


In [3]:
env = gym.make('CartPole-v1')

[2017-06-25 21:59:04,563] Making new env: CartPole-v1


In [4]:
class DeepQ:
    def __init__(self,na,ns):
        self.ns = ns
        self.na = na
        self.store = []
        self.episode = 1000
        self.discount = 1
        self.epsilon = 1
        self.learning_rate = 0.001
        self.model = self._model()
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
    
    def _model(self):
        model = Sequential()
        model.add(Dense(10,input_shape=(self.ns,),activation='relu'))
        model.add(Dense(16,activation='relu'))
        model.add(Dense(self.na,activation='softmax'))
        model.compile(optimizer=Adam(lr=self.learning_rate),loss='mean_squared_error')
#         model = Sequential()
#         model.add(Dense(24, input_shape=(self.ns,), activation='relu'))
#         model.add(Dense(24, activation='relu'))
#         model.add(Dense(self.na, activation='linear'))
#         model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model
    
    def memory(self,step):
        self.store.append((step[0],step[1],step[2],step[3],step[4])) #state,action,next_state,reward,done
    
#     def soft_greedy(self):
#         def policy_fn(observation):
#             A = np.ones(self.na, dtype=float) * self.epsilon / self.na
# #             best_action = np.argmax(Q[observation])
#             best_action = np.argmax(self.model.predict(observation)[0])
#             A[best_action] += (1.0 - self.epsilon)
#             return A
#         return policy_fn
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.na)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def greedy(self, nA):
        def policy_fn(observation):
            A = np.zeros(nA, dtype=float)
            best_action = np.argmax(self.model.predict(observation)[0])
            A[best_action] = 1
            return A
        return policy_fn
    
    def expirence_replay(self,batch_size):
#         pass
        batch = random.sample(self.store,batch_size)
        for s,a,s_,R,done in batch:
            temp = R + self.discount * np.amax(self.model.predict(s_)[0]) if not done else R
            target = self.model.predict(s)
            target[0][a] = temp
            self.model.fit(s,target,verbose=0,epochs=1)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [5]:
deepQ = DeepQ(env.action_space.n,env.observation_space.shape[0])
# policy = deepQ.soft_greedy(env.action_space.n)
batch_size = 40

for i_episode in xrange(deepQ.episode):
    s = env.reset()
    s = np.reshape(s, [1, deepQ.ns])
    for i in itertools.count():
#         probs = policy(s)
#         a = np.random.choice(np.arange(len(probs)),p=probs)
#         s_,R,done,_ = env.step(a)
        a = deepQ.act(s)
        s_,R,done,_ = env.step(a)
        s_ = np.reshape(s_, [1, deepQ.ns])
        R = R if not done else -5
        deepQ.memory((s,a,s_,R,done))
        if done:
            break
        s = s_
        
    if batch_size < len(deepQ.store):
        deepQ.expirence_replay(batch_size)
    
    
    if (i_episode+1)%100 == 0:
        print i_episode+1
    
# deepQ.save('cartpole.h5')

100
200
300
400
500
600
700
800
900
1000


In [6]:
# polcy = soft_deepQ.greedy()
reward_list = []
for j_episode in xrange(10):
    s = env.reset()
    s = np.reshape(s,[1,deepQ.ns])
    r = 0
    for j in itertools.count():
#         probs = polcy(s)
#         a = np.random.choice(np.arange(len(probs)),p=probs)
        s_,R,done,_ = env.step(deepQ.act(s))
#         s_,R,done,_ = env.step(a)
#         s_,R,done,_ = env.step(0)
        s_ = np.reshape(s_,[1,deepQ.ns])
        r += R
        if done:
            break
        s = s_
    reward_list.append(r)

print reward_list

[8.0, 8.0, 10.0, 10.0, 9.0, 8.0, 11.0, 10.0, 10.0, 10.0]


In [ ]:
env.observation_space.sample()

In [ ]:
np.reshape(env.observation_space.sample(), [1, env.observation_space.shape[0]])

In [ ]:
env.action_space.n

In [ ]:
abcd = [[1,1],[2,2],[3,3],[4,4],[5,5],[6,6],[7,7],[8,8]]
random.sample(abcd,k=2)


In [ ]:
l = []
for i in xrange(10):
    l.append(env.observation_space.sample())
for i in l:
    i = np.reshape(i,[1,deepQ.ns])
    print i
    print deepQ.act(i)